In [1]:
import numpy as np
import pathlib
from PIL import Image
import re
import joblib
import torch
import torchvision
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
pop_directory = "Data/pop"
rock_directory = "Data/rock"
country_directory = "Data/country"
rock_pop_directory = "Data/rock_pop"

save_pop = "assembled_pop"
save_rock = "assembled_rock"
save_country = "assembled_country"
save_rock_pop = "assembled_rock_pop"

filename_gen = "SD3_v3/GeneratorSD3_v3.joblib"
generator = joblib.load(filename_gen)


In [3]:
# class MyImageFolder(torch.utils.data.Dataset):
#     def __init__(self, directory:str, transform=None):
#
#         self.paths = list(pathlib.Path(directory).glob('*/*.jpg'))
#
#         self.transform = transform
#
#     def __len__(self):
#         return len(self.paths)
#
#     def __getitem__(self, index):
#         img = self.load_image(index)
#         class_name = self.paths[index].parent.name
#         class_index = self.class_to_index[class_name]
#
#         if self.transform:
#             return self.transform(img), class_index
#         else:
#             return img, class_index
#
#     def load_image(self, index):
#         image_path = self.paths[index]
#         return Image.open(image_path)
#
# transform = torchvision.transforms.Compose([
#             torchvision.transforms.ToTensor(),  # imaginile sunt normalizate intre [0, 1]
#         ])
#


In [20]:
def load_image(paths, index):
    image_path = paths[index]
    # print(image_path)
    return Image.open(image_path)

def assemble(about, paths, save, genre):
    print("In assemble")
    for ind, i in enumerate(about):
        # assemble the i-th spectrogram
        total_width = i[0]
        s_per_line = total_width // 32
        extra = 0
        if total_width % 32 != 0:  # daca latimea nu se imparte la 32 inseamna ca mai avem o bucata incompleta la final
            extra = 1

        final_spect = Image.new('L', (total_width, 128))
        counter = i[1][0]
        for j in range(4):
            # j numara liniile
            for k in range(s_per_line):
                # k numara coloanele
                # incarcam cate o bucata si o punem pe pozitia ei
                spect = load_image(paths, counter)
                final_spect.paste(spect, (k * 32, j * 32))
                counter += 1
            if extra:
                spect = load_image(paths, counter)
                counter += 1
                remaining = total_width % 32
                spect_np = np.array(spect)
                last_columns = spect_np[:, -remaining:]
                spect = Image.fromarray(last_columns)
                final_spect.paste(spect, (total_width // 32, j * 32))

        final_spect.save(f"Data/{save}/{genre}_spec{ind}.jpg")
        if counter != i[1][1] : print(counter, i[1][1])


def transform_spects(paths, genre):
    print("In transform")
    for ind in tqdm(range(len(paths))):
        image = load_image(paths, ind)
        if ind == 0:
            fig, axs = plt.subplots(1)
            axs.imshow(image)
            plt.show()

        make_tensor = torchvision.transforms.ToTensor()
        image = make_tensor(image)

        if ind == 0:
            image = torch.transpose(image, 0, 2)
            fig, axs = plt.subplots(1)
            axs.imshow(image)
            plt.show()
            image = torch.transpose(image, 0, 2)

        image = torch.unsqueeze(image, 0)
        image = generator(image)
        # daca nu merge fac cu slice

        image = torch.squeeze(image, 0)
        if ind == 0:
            image = torch.transpose(image, 0, 2)
            fig, axs = plt.subplots(1)
            axs.imshow(image.detach())
            plt.show()
            image = torch.transpose(image, 0, 2)

        # image_arr = np.array(image)
        # image_tensor = torch.from_numpy(image_arr)
        # image_tensor = image_tensor.to(torch.float32)
        # image_tensor = torch.unsqueeze(image_tensor, 0)
        # image_tensor = torch.unsqueeze(image_tensor, 0)
        # new_image = generator(image_tensor)
        # new_image = torch.squeeze(new_image)

        #image = torch.transpose(image, 0, 2)
        image = torch.squeeze(image, 0)

        new_np = image.detach().numpy().astype(np.float32)
        new_np = (new_np * 255).astype(np.uint8)
        if ind == 0:
            fig, axs = plt.subplots(1)
            axs.imshow(new_np)
            plt.show()

        im = Image.fromarray(new_np)
        if im.mode != 'L':
            im = im.convert('L')
        im.save(f"Data/{genre}/{genre}_spec{ind}.jpg")


about_pop = np.load('pop_items.npy', allow_pickle=True)
#about_rock = np.load('rock_items.npy', allow_pickle=True)
about_country = np.load('country_items.npy', allow_pickle=True)

pop_paths = sorted(pathlib.Path(pop_directory).glob('*.jpg'), key=lambda p: int(re.search(r'\d+', p.stem).group()))
#rock_paths = sorted(pathlib.Path(rock_directory).glob('*.jpg'), key=lambda p: int(re.search(r'\d+', p.stem).group()))
country_paths = sorted(pathlib.Path(country_directory).glob('*.jpg'), key=lambda p: int(re.search(r'\d+', p.stem).group()))

#transform_spects(rock_paths, 'rock_pop')

#rock_pop_paths = sorted(pathlib.Path(rock_pop_directory).glob('*.jpg'), key=lambda p: int(re.search(r'\d+', p.stem).group()))

assemble(about_pop, pop_paths, save_pop, 'pop')
#assemble(about_rock, rock_paths, save_rock, 'rock')
assemble(about_country, country_paths, save_country, 'country')
#assemble(about_rock, rock_pop_paths, save_rock_pop, 'rock_pop')

In assemble
In assemble
